In [11]:
class Issue:
    def __init__(self, title, description, url, featured_answer):
        self.title = title
        self.description = description
        self.url = url
        self.featured_answer = featured_answer

issues_batch = []

In [18]:
# get repos
# iterate through repos' issues(using next/prev link headers)
# iterata through issues' comments and get featured comment by properties : ["author_association"]
# create list of issues objects
# save to parquet
# vecotrize issues and save to vector database

import requests
import dotenv
import os
import pandas as pd

dotenv.load_dotenv()
baseurl = "https://api.github.com/"
token = os.getenv("GH_PAT")
api_version = "2022-11-28"
headers = {
    "Accept": "application/vnd.github.v3+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": api_version
}

api = "repositories"
repos_link = f"{baseurl}{api}"
iterations = 0
while repos_link:
    resp = requests.get(repos_link, headers=headers)
    public_repos = resp.json()
    link_header = resp.headers.get("Link")
    if "rel=\"next\"" in link_header: 
        repos_link = link_header.split(",")[0].split(";")[0].replace("<",'').replace(">",'')
    else:
        repos_link = None
    #testing 
    public_repos = public_repos[3:4]
    #-------
    for repo in public_repos:
        print(f"Processing {repo['full_name']}")
        issues_link = f"{baseurl}repos/{repo['full_name']}/issues"
        while issues_link:
            print(f"Processing {issues_link}")
            resp = requests.get(issues_link, headers=headers)
            repo_issues = resp.json()
            link_header = resp.headers.get("Link")
            try:
                issues_links = link_header.split(",")
                issues_link = next((link.split(';')[0].replace("<",'').replace(">",'') for link in issues_links if "rel=\"next\"" in link), None)
            except:
                issues_link = None
            for issue in repo_issues:
                issue_comments = requests.get(f"{baseurl}repos/{repo['full_name']}/issues/{issue['number']}/comments?sort=created&direction=asc", headers=headers).json()
                featured_comment = None
                # should we just get one answer or contact multiple
                for comment in issue_comments:
                    if (comment["author_association"] == "OWNER"):
                        featured_comment = comment["body"]
                        break
                    elif (comment["author_association"] == "COLLABORATOR"):
                        featured_comment = comment["body"]
                        break
                    elif (comment["author_association"] == "MEMBER"):
                        featured_comment = comment["body"]
                        break
                    elif (comment["author_association"] == "CONTRIBUTOR"):
                        featured_comment = comment["body"]
                        break
                if featured_comment and issue["body"]:
                    issues_batch.append(Issue(issue["title"], issue["body"], issue["url"], featured_comment))
    iterations += 1
    with open("./logs/issues-gh.log", "a") as f:
        f.write(f"Next Page: {repos_link} .Processed {iterations} iterations\n")
    df = pd.DataFrame([vars(issue) for issue in issues_batch], columns=["title", "description", "url", "featured_answer"], dtype=object)
    print(df.head())
    df.to_parquet(f"./output/issues-gh-{iterations}.parquet")


Processing mojombo/god
Processing https://api.github.com/repos/mojombo/god/issues
Processing https://api.github.com/repositories/28/issues?page=2
Processing  https://api.github.com/repositories/28/issues?page=3
Processing  https://api.github.com/repositories/28/issues?page=4
                                               title  \
0               Object#timeout is deprecated warning   
1               Object#timeout is deprecated warning   
2  Explicitly catch NoMethodError instead of catc...   
3                     Use simplecov instead of rcov.   
4           Hipchat integration requires admin token   

                                         description  \
0  I received the following warning when God star...   
1  I received the following warning when God star...   
2  This can be replicated by loading a watch, sto...   
3  This PR replaces rcov with simplecov, which su...   
4  From the `god` docs:\n\n```\nNote: in Hipchat ...   

                                                 u

KeyboardInterrupt: 

In [ ]:
# search for poststs in stackoverflow
# get featured answer by properties : ["is_accepted", "score"]
# create list of issues objects
# save all to parquet
# vecotrize posts with is_answered: true and save to vector database